<a href="https://colab.research.google.com/github/Alexandre-Data/OC-Projet-7/blob/main/Projet7_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!git version
!pip install mlflow
!pip install SMOTE
!pip install -U imbalanced-learn
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
import smote

import mlflow
import mlflow.xgboost
from joblib import dump, load

Mounted at /content/drive
git version 2.34.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none-any.whl size=143861 sha256=5f55956e6fd19fc6f40f271712dddeb14bb62e5fbd100a2bc19b691b12a97e4c
  Stored in directory: /root/.cache/pip/wheels/ae/63/93/5402c1a09c1868a59d0b05013484e07af97a9d7b3dbd

In [2]:
sqlite_uri = "/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/my_mlflow_data.db"

In [3]:
tracking_uri = f"sqlite:///{sqlite_uri}"
mlflow.set_tracking_uri(tracking_uri)

In [4]:
mlflow.set_experiment("Projet 7")

<Experiment: artifact_location='/content/mlruns/1', creation_time=1691574488173, experiment_id='1', last_update_time=1691574488173, lifecycle_stage='active', name='Projet 7', tags={}>

In [5]:
df=pd.read_csv("/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/dfclean_targetnan.csv")

In [6]:
df.shape

(334329, 493)

In [7]:
df['TARGET'].isna().sum()

47208

In [8]:
df.isna().sum().sum()

47208

In [ ]:
# Définir la fonction de perte personnalisée
def custom_loss(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fn_weight = 10.0  # poids pour les faux négatifs
    fp_weight = 10.0  # poids pour les faux positifs
    return cm[0, 1] * fp_weight + cm[1, 0] * fn_weight

In [ ]:
# Faire une fonction d'évaluation personnalisée pour GridSearch
custom_scorer = make_scorer(custom_loss, greater_is_better=False)

In [ ]:
# Paramètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Création du modèle de forêt aléatoire
model_rf_opti = RandomForestClassifier()

# Création du GridSearch
grid_search_rf_opti = GridSearchCV(estimator=model_rf_opti, param_grid=param_grid,
                                   scoring=custom_scorer, cv=3, n_jobs=-1)

In [ ]:
# Séparer les données de validation
validation_data = df[df['TARGET'].isna()]
X_validation = validation_data.drop(columns=['TARGET'])
y_validation = validation_data['TARGET']

# Séparer les données d'entraînement et de test
train_test_data = df.dropna(subset=['TARGET'])
X = train_test_data.drop(columns=['TARGET'])
y = train_test_data['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Créer le pipeline avec SMOTE et GridSearchCV
pipeline = make_pipeline(SMOTE(), grid_search_rf_opti)

# Démarrer une expérience MLflow
with mlflow.start_run() as run:
    mlflow.set_tag("model_version", "FORET ALEA OPTI")
    # Adapter le modèle
    pipeline.fit(X_train, y_train)

    # Prédire sur l'ensemble de test
    y_pred = pipeline.predict(X_test)

    # Calculer les métriques
    accuracy_rf_base = accuracy_score(y_test, y_pred)
    precision_rf_base = precision_score(y_test, y_pred)
    recall_rf_base = recall_score(y_test, y_pred)
    f1_rf_base = f1_score(y_test, y_pred)
    roc_auc_rf_base = roc_auc_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    # Loguer les métriques dans MLflow
    mlflow.log_metric("accuracy", accuracy_rf_base)
    mlflow.log_metric("precision", precision_rf_base)
    mlflow.log_metric("recall", recall_rf_base)
    mlflow.log_metric("f1_score", f1_rf_base)
    mlflow.log_metric("roc_auc_score", roc_auc_rf_base)

    # Afficher les scores
    print(f'Accuracy: {accuracy_rf_base*100:.2f}%')
    print(f'Precision: {precision_rf_base*100:.2f}%')
    print(f'Recall: {recall_rf_base*100:.2f}%')
    print(f'F1-Score: {f1_rf_base*100:.2f}%')
    print(f'ROC AUC Score: {roc_auc_rf_base*100:.2f}%')
    print(f'Confusion Matrix: {cm}')


In [ ]:
# Sauvegarder le modèle dans un fichier joblib
model_filepath = "/content/drive/MyDrive/OPENCLASSROOM/PROJET 7/model_f_alea_opti.joblib"
dump(pipeline, model_filepath)

print(f'Model saved to {model_filepath}')